In [ ]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
import geemap
import ee
Map=geemap.Map()
Map 

In [ ]:
roi = ee.Geometry.Rectangle([14.495753, 51.355146, 14.59681, 51.400062])
aoi = ee.Feature(roi)
Map.addLayer(aoi,{},'aoi')
Map.centerObject(aoi,12)
print(aoi.getInfo())

In [ ]:
def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)
    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# Return the DN that maximizes interclass variance in B5 (in the region).
def otsu(image):
    histogram = image.select('VV').reduceRegion(**{
        'reducer': ee.Reducer.histogram(255, 2),
        'geometry': aoi.geometry(), 
        'scale': 30,
        'bestEffort': True
    })
    counts = ee.Array(ee.Dictionary(histogram.get('VV')).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram.get('VV')).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
    indices = ee.List.sequence(1,size)
    # Compute between sum of squares, where each mean partitions the data.
    bss = indices.map(lambda i : sumSquare(i,histogram.get('VV'),counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return image.set({'th':means.sort(bss).get([-1])})

In [ ]:
# Temporal smoothing of every image given list of images to use for search and time window in days
def TemporalSmoothing(image):
    t0 = image.date()
    tStart = t0.advance(-20, 'day')
    tStop = t0.advance(20, 'day')
    return aimages.filterDate(tStart, tStop).median().copyProperties(image).copyProperties(image, ['system:time_start'])

# Compute surface water area by calling Otsu thresholding and taking darker mode
def computeWaterArea(image):
    water = image.lt(ee.Number(image.get('th')))
    areas = water.selfMask().multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': aoi.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas})

In [ ]:
# get images
aimages = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2018-01-01','2019-01-01') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filterBounds(aoi.geometry())
print(aimages.size().getInfo())
Map.addLayer(aimages.first(),{},'first')

In [ ]:
# print(computeWaterArea(otsu(aimages.first())).get('area').getInfo())

In [ ]:
times = aimages.aggregate_array('system:time_start')
imagesSmoothed = aimages.map(TemporalSmoothing)
valuesRaw = aimages.select('VV').map(otsu).map(computeWaterArea).aggregate_array('area')
valuesSmoothed = imagesSmoothed.select('VV').map(otsu).map(computeWaterArea).aggregate_array('area')

# print(valuesRaw.getInfo())
# print(valuesSmoothed.getInfo())

In [ ]:
def func_xsr(i):
    q = ee.List(i).flatten()
    return ee.Feature(None,{'time': q.get(0),'area': ee.Dictionary(q.get(1)).get('VV'),'area_smoothed': ee.Dictionary(q.get(2)).get('VV')})
chartValues = times.zip(valuesRaw).zip(valuesSmoothed).map(func_xsr)

In [ ]:
print(ee.Date('2020-01-01').getInfo())
print(ee.Date('2020-01-01').getInfo()['value'])

In [ ]:
ab = ee.Feature(None,{'time': 0,'area': 1,'area_smoothed': 2})
print(ab.getInfo())